In [58]:
import torch
from argparse import ArgumentParser
from pprint import pprint
from tqdm import tqdm
import time

from arguments import ModelParams, PipelineParams, OptimizationParams

parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)
op = OptimizationParams(parser)
pp = PipelineParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--test_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--save_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[])
parser.add_argument("--start_checkpoint", type=str, default = None)

args = parser.parse_args(args=[
    "--source_path", "/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train/",
])
pprint(vars(lp.extract(args)))

print()
print(type(lp), type(lp.extract(args)))
print(args.resolution)
print()

pprint(vars(lp))

{'data_device': 'cuda',
 'eval': False,
 'images': 'images',
 'model_path': '',
 'resolution': -1,
 'sh_degree': 3,
 'source_path': '/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train',
 'white_background': False}

<class 'arguments.ModelParams'> <class 'arguments.GroupParams'>
-1

{'_images': 'images',
 '_model_path': '',
 '_resolution': -1,
 '_source_path': '',
 '_white_background': False,
 'data_device': 'cuda',
 'eval': False,
 'sh_degree': 3}


In [30]:
(
    active_sh_degree,
    _xyz,
    _features_dc,
    _features_rest,
    _scaling,
    _rotation,
    _opacity,
    max_radii2D,
    xyz_gradient_accum,
    denom,
    state_dict,
    spatial_lr_scale,
), saved_iter = torch.load(
    "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting-original/output/22ef02fc-3/chkpnt10.pth"
)

In [34]:
_features_rest.mean()

tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)

In [7]:
# read camera extrinsics
import os

from scene.dataset_readers import readCamerasFromTransforms
from scene.colmap_loader import (
    read_extrinsics_binary,
    read_intrinsics_binary,
)

SRC_PATH = "/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train"
cameras_extrinsic_file = os.path.join(SRC_PATH, "sparse/0", "images.bin")
cameras_intrinsic_file = os.path.join(SRC_PATH, "sparse/0", "cameras.bin")
cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)

In [56]:
cam_extrinsics[1].xys.shape

(11661, 2)

In [35]:

import numpy as np
import collections
import struct

CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple(
    "Camera", 
    ["id", "model", "width", "height", "params"]
)
BaseImage = collections.namedtuple(
    "Image",
    [
        "id", "qvec", "tvec", "camera_id",
        "name", "xys", "point3D_ids"
    ]
)
Point3D = collections.namedtuple(
    "Point3D",
    [
        "id", "xyz", "rgb", "error",
        "image_ids", "point2D_idxs"
    ]
)

CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12)
}
CAMERA_MODEL_IDS = dict([(camera_model.model_id, camera_model)
                         for camera_model in CAMERA_MODELS])
CAMERA_MODEL_NAMES = dict([(camera_model.model_name, camera_model)
                           for camera_model in CAMERA_MODELS])


class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)
    

def qvec2rotmat(qvec):
    return np.array([
        [
            1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
            2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
            2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]
        ],
        [
            2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
            1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
            2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]
        ],
        [
            2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
            2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
            1 - 2 * qvec[1]**2 - 2 * qvec[2]**2
        ]
    ])


In [2]:

import torch
from argparse import ArgumentParser
from pprint import pprint
from tqdm import tqdm
import time

from arguments import ModelParams, PipelineParams, OptimizationParams

# test GaussianModel
from scene.gaussian_model import GaussianModel
from scene import Scene

parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)
op = OptimizationParams(parser)
pp = PipelineParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--test_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--save_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[])
parser.add_argument("--start_checkpoint", type=str, default = None)
args = parser.parse_args(args=[
    "--source_path", "/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train/",
])

model_args = lp.extract(args)

gaussian_model = GaussianModel(sh_degree=100)
scene = Scene(model_args, gaussian_model)


AssertionError: Could not recognize scene type!

In [ ]:

import torch
from argparse import ArgumentParser
from pprint import pprint
from tqdm import tqdm
import time

from arguments import ModelParams, PipelineParams, OptimizationParams

# test GaussianModel
from scene.gaussian_model import GaussianModel
from scene import Scene

parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)
op = OptimizationParams(parser)
pp = PipelineParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--test_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--save_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[])
parser.add_argument("--start_checkpoint", type=str, default = None)
args = parser.parse_args(args=[
    "--source_path", "/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train/",
])

model_args = lp.extract(args)

gaussian_model = GaussianModel(sh_degree=100)
scene = Scene(model_args, gaussian_model)


/home/hjp/.conda/envs/gs/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading camera 301/301
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  182686


RuntimeError: CUDA out of memory. Tried to allocate 20.83 GiB (GPU 0; 23.69 GiB total capacity; 22.64 GiB already allocated; 47.81 MiB free; 22.80 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF